https://mmuratarat.github.io/2019-01-17/implementing-padding-schemes-of-tensorflow-in-python

In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'2.0.0'

In [2]:
class CustomConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides, padding, 
                 kernel_initializer, bias_initializer, pad_type='CONSTANT', **kwargs):
        '''
        filters - int, number of filters (channels_out)
        kernel_size - tuple (kernel_height, kernel_width)
        strides - tuple (stride_height, stride_width)
        padding - 'SAME' or 'VALID'
        pad_type - 'CONSTANT' (zeros) or 'REFLECT'
        '''
        super(CustomConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = np.array(kernel_size)
        self.strides = np.array(strides)
        self.padding = padding.upper()
        self.pad_type = pad_type.upper()
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        
    
    def build(self, input_shape):
        '''
        input_shape = (batch_size, height, width, channels_in)
        '''
        _, self.height, self.width, self.channels_in = input_shape.as_list()
        self.img_size = np.array([self.height, self.width])
        
        self.kernel = self.add_weight("kernel", 
                                      shape=(self.kernel_size[0], self.kernel_size[1], self.channels_in, self.filters), 
                                      initializer=self.kernel_initializer)
        self.bias = self.add_weight("bias", shape=(self.filters,), initializer=self.bias_initializer)
        super(CustomConv2D, self).build(input_shape)  # Be sure to call this at the end
    
    
    def call(self, inputs):
        if self.padding == 'VALID': # no padding
            x = tf.nn.conv2d(inputs, self.kernel, self.strides, 'VALID')
        elif self.padding == 'SAME':
            # padding = [pad_height, pad_width]
            padding = np.where(self.img_size % self.strides == 0, 
                               np.maximum(self.kernel_size - self.strides, 0), 
                               np.maximum(self.kernel_size - self.img_size % self.strides, 0))
            pad_top, pad_left = padding // 2
            pad_bottom, pad_right = padding - padding // 2
            
            x = tf.pad(inputs, [[0, 0], [pad_top, pad_bottom], [pad_left, pad_right], [0, 0]], mode=self.pad_type)
            x = tf.nn.conv2d(x, self.kernel, self.strides, 'VALID')
        x = tf.nn.bias_add(x, self.bias)
        return x

In [3]:
x = np.random.rand(2, 13, 18, 21)
filters = 11
kernel_size = (5,5)
strides = (4,4)
padding = 'same'
kernel_initializer = tf.constant_initializer(2.0)
bias_initializer = tf.constant_initializer(1.5)

In [4]:
y1 = CustomConv2D(filters, kernel_size, strides, padding, kernel_initializer, bias_initializer, name='custom_conv')(x)
y2 = tf.keras.layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, 
                            padding=padding, kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)(x)

W1226 10:16:12.389894 16600 base_layer.py:1814] Layer custom_conv is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

W1226 10:16:12.517030 16600 base_layer.py:1814] Layer conv2d is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you i

In [5]:
np.array_equal(y1.numpy(), y2.numpy())

True